In [2]:
!pip install ray

     ---------------------------------------- 20.3/20.3 MB 4.6 MB/s eta 0:00:00
  Using cached grpcio-1.43.0-cp39-cp39-win_amd64.whl (3.4 MB)
     ---------------------------------------- 62.5/62.5 kB ? eta 0:00:00
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
  Using cached platformdirs-2.5.2-py3-none-any.whl (14 kB)
  Using cached distlib-0.3.4-py2.py3-none-any.whl (461 kB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.3
    Uninstalling grpcio-1.46.3:
      Successfully uninstalled grpcio-1.46.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.3 requires wrapt~=1.12.1, which is not installed.
sacremoses 0.0.45 requires regex, which is not installed.
pytorch-transformers 1.2.0 requires regex, which is not installed.
tensorflow 2.5.3 requires grpcio~=1.34.0, but you have grpcio 1.43.0 which is incompatible.
tensorflow 2.5.3 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.5.3 requires typing-extensions~=3.7.4, but you have typing-extensions 4.2.0 which is incompatible.


In [3]:
import ray

In [5]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
#from fbprophet import Prophet
import pickle
import copy

In [7]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: total: 797 ms
Wall time: 798 ms


In [6]:
%%time
''' 스무딩 '''
win_size = 3


for plc in plc_lst:
    df = df_dic[plc]
    df_dic[plc] = df_dic[plc].fillna(method='ffill')
    df_dic[plc] = df_dic[plc].fillna(method='bfill')
    df_dic[plc] = df_dic[plc].dropna()
    df_dic[plc] = df_dic[plc].iloc[-1000:,:]

CPU times: user 3.86 s, sys: 3.55 ms, total: 3.86 s
Wall time: 3.87 s


df = df_dic[plc_lst[0]]
sns.lineplot(x = df.ds , y = df.pH)

df.pH.iloc[2800:]

In [7]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

In [38]:
@ray.remote
def pred_all_columns(plc_df,i,fct_lst,submit):
    
    res_list = []
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        model = Prophet()
        model.fit(fct_df)
        forecast = model.predict(future)
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast = forecast[['ds', 'yhat']]
        
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
            
        res_list.append( (i,j,forecast) )
    return res_list

In [39]:
fct_lst_ray = ray.put(fct_lst)
submit_ray = ray.put(submit)

In [ ]:
task_list = [ pred_all_columns.remote(df_dic[x], x ,fct_lst_ray, submit_ray) for x in plc_lst ]

In [ ]:
%%time
results = ray.get(task_list)

In [49]:
for item in results:
    for x in item:
        i = x[0]
        j = x[1]
        forecast = x[2]
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])

/tmp/ipykernel_10600/3712719606.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])


In [50]:
# 추론 파일 저장
with open('./sub/baseline_median3.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)